In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
import tensorflow as tf

In [5]:
sess = tf.InteractiveSession #式の作成と実行を交互に行うのでこれを指定する

In [7]:
#重み。行列を渡すと初期化して返す
def weight_variable(shape):
    #truncated_normal:正規分布の左右を切り取ったもののイメージ
    #stddev:標準偏差
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [8]:
#バイアス
def bias_variable(shape):
    #0だと計算が進まないのでバイアス値は少し
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [11]:
#たたみ込み処理の定義
def conv2d(x, W):
    #strides = [1,1,1,1]:1pix毎に処理する事
    #padding = 'SAME':左右に0を足したものにする
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')

In [13]:
#たたみ込んだ画像をsizeを小さくして特徴を抽出。一番sizeの大きなデータをかえす
def max_pool_2x2(x):
    #ksize適用数範囲2*2
    #strides 2*2ずらして処理
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [14]:
#1層目の重みの初期化　５＊５の画像に１このデータが３２枚
W_conv1 = weight_variable([5,5,1,32])

In [17]:
#bias
b_conv1 = bias_variable([32])

In [ ]:
#-1は実行時に自動的に値を設定してくれるらしい
x_image = tf.reshape(x, [-1,28,28,1])